In [47]:
import os
import sys
import urllib.request
from io import BytesIO
from zipfile import ZipFile

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dbfread import DBF
from IPython.display import display

# Google OR tools
from ortools.linear_solver import pywraplp

pd.set_option("display.max_columns", None)

## Importing Data


### Field Data


In [48]:
# Importing the field_dataset from the csv file
filepath = "https://raw.githubusercontent.com/percw/Norwegian_oil_gas_decarbonization/main/data/output/emissions_and_production/cleaned/fields_prod_emissions_1997_2023.csv"

# Creating a check if import is successful
try:
    field_data = pd.read_csv(filepath, sep=",")
    print("Data import successful")
    field_data = field_data.rename(
    columns={
        "share_intensity_tco2e/toe_gwp100": "emission_intensity_distributed",
        "kgco2e/toe_int_gwp100": "emission_intensity",
    }
    )   

    # Year is %Y datetime
    field_data["year"] = pd.to_datetime(field_data["year"], format="%Y")
except:
    print("field_data import failed")
    



Data import successful


### Geolocations


In [49]:
# Define the URLs for the shapefile components
base_url = "https://github.com/percw/Norwegian_oil_gas_decarbonization/tree/main/data/raw_data/geo/fields/"
shapefile_components = ["fldArea.shp", "fldArea.shx", "fldArea.dbf"]

# Local directory to save downloaded files
local_dir = "../../data/raw_data/geo/fields"

# Function to download a file from a URL to a local path
def download_file(url, local_path):
    urllib.request.urlretrieve(url, local_path)

# Download the shapefile components if they do not exist locally
if not os.path.exists(local_dir):
    os.makedirs(local_dir)
    for component in shapefile_components:
        local_path = os.path.join(local_dir, os.path.basename(component))
        download_file(base_url + component, local_path)
else:
    print("Field data already exists")

# Function to read shapefile into GeoPandas
def import_field_data():
    fields = gpd.read_file(os.path.join(local_dir, "fldArea.shp"))
    return fields

# Import field data
fields = gpd.read_file('../../data/raw_data/geo/fields/fldArea.shp')

# Now, `fields` contains the GeoDataFrame with the field data
print(fields.head())

Field data already exists
          fieldName curActStat discWelNam  discYear  \
0        ALBUSKJELL  Shut down      1/6-1      1972   
1  TOMMELITEN GAMMA  Shut down      1/9-4      1978   
2              VARG  Shut down    15/12-4      1984   
3     SLEIPNER VEST  Producing     15/6-3      1974   
4            GUNGNE  Producing    15/9-15      1982   

                  owner_kind          owner_name  idOwner  \
0         PRODUCTION LICENSE                 018    20900   
1  BUSINESS ARRANGEMENT AREA     TOMMELITEN UNIT    41411   
2                        NaN                 NaN        0   
3  BUSINESS ARRANGEMENT AREA  SLEIPNER VEST UNIT    40848   
4         PRODUCTION LICENSE                 046    21156   

                      OpLongName  idCompany fldSupBase          Dctype  \
0  ConocoPhillips Skandinavia AS    2410696        NaN  GAS/CONDENSATE   
1  ConocoPhillips Skandinavia AS    2410696        NaN  GAS/CONDENSATE   
2                            NaN          0   Tananger

In [50]:
# Convert fieldName to lowercase
fields["fieldName"] = fields["fieldName"].str.lower()
fields.head(3)

,fieldName,curActStat,discWelNam,discYear,owner_kind,owner_name,idOwner,OpLongName,idCompany,fldSupBase,Dctype,main_area,discWelUrl,MapUrl,FactUrl,idWellbore,idField,dtUpdated,dtUpdMax,geometry
0,albuskjell,Shut down,1/6-1,1972,PRODUCTION LICENSE,018,20900,ConocoPhillips Skandinavia AS,2410696,NaN,GAS/CONDENSATE,North sea,https://factpages.sodir.no/factpages/default.a...,https://factmaps.sodir.no/factmaps/3_0/?run=Fi...,https://factpages.sodir.no/factpages/default.a...,239,43437,2024-01-02,2024-01-02,"POLYGON ((2.90643 56.65801, 2.90645 56.65801, ..."
1,tommeliten gamma,Shut down,1/9-4,1978,BUSINESS ARRANGEMENT AREA,TOMMELITEN UNIT,41411,ConocoPhillips Skandinavia AS,2410696,NaN,GAS/CONDENSATE,North sea,https://factpages.sodir.no/factpages/default.a...,https://factmaps.sodir.no/factmaps/3_0/?run=Fi...,https://factpages.sodir.no/factpages/default.a...,247,43444,2024-01-02,2024-03-01,"POLYGON ((2.92952 56.47679, 2.91982 56.47901, ..."
2,varg,Shut down,15/12-4,1984,NaN,NaN,0,NaN,0,Tananger,OIL,North sea,https://factpages.sodir.no/factpages/default.a...,https://factmaps.sodir.no/factmaps/3_0/?run=Fi...,https://factpages.sodir.no/factpages/default.a...,438,43451,2024-04-16,2024-01-02,"MULTIPOLYGON (((1.94008 58.04863, 1.93786 58.0..."


### Prediction Data


In [51]:
# Importing the field_dataset from the csv file
filepath_pred = "https://raw.githubusercontent.com/percw/Norwegian_oil_gas_decarbonization/main/data/output/emissions_and_production/cleaned/field_predictions.csv"

# Creating a check if import is successful
try:
    field_pred_data = pd.read_csv(filepath_pred, sep=",")
    print("Data import successful")
except:
    print("Data import failed")

field_pred_data.head(3)


Data import successful


,field,year,predicted_production,cumulative_production,total_emissions,years_left,reserve,mean_emission_intensity_distributed,electrified
0,statfjord nord,2024,1.542401,1.542401,156151.785958,3.0,4.050000,336.019819,0
1,statfjord nord,2025,1.542401,3.084802,158096.887669,2.0,2.507599,336.019819,0
2,statfjord nord,2026,0.612391,3.697193,65653.808262,1.0,0.965198,336.019819,0


### Data Merging


In [52]:
# Adding the geometry to the field_pred_data 

field_pred_data = field_pred_data.merge(fields[["geometry", "fieldName"]], left_on="field", right_on="fieldName", how="left")
field_pred_data.head(3)

,field,year,predicted_production,cumulative_production,total_emissions,years_left,reserve,mean_emission_intensity_distributed,electrified,geometry,fieldName
0,statfjord nord,2024,1.542401,1.542401,156151.785958,3.0,4.050000,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4...",statfjord nord
1,statfjord nord,2025,1.542401,3.084802,158096.887669,2.0,2.507599,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4...",statfjord nord
2,statfjord nord,2026,0.612391,3.697193,65653.808262,1.0,0.965198,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4...",statfjord nord


In [53]:
# Drop fieldName column
field_pred_data = field_pred_data.drop(columns=["fieldName"])
field_pred_data.head(3)

,field,year,predicted_production,cumulative_production,total_emissions,years_left,reserve,mean_emission_intensity_distributed,electrified,geometry
0,statfjord nord,2024,1.542401,1.542401,156151.785958,3.0,4.050000,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4..."
1,statfjord nord,2025,1.542401,3.084802,158096.887669,2.0,2.507599,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4..."
2,statfjord nord,2026,0.612391,3.697193,65653.808262,1.0,0.965198,336.019819,0,"MULTIPOLYGON (((1.99934 61.46574, 2.00438 61.4..."


## Optimization


In [54]:
# Ensuring all fields have entries for every year from 2024 to 2050
fields_to_fill = field_pred_data['field'].unique()
years = list(range(2024, 2050))

# Create a DataFrame with all combinations of fields and years
all_combinations = pd.MultiIndex.from_product([fields_to_fill, years], names=["field", "year"]).to_frame(index=False)

# Merge with the original data to fill in missing combinations with zero production and zero emissions
field_pred_data_complete = all_combinations.merge(field_pred_data, on=["field", "year"], how="left")

# Forward fill the fields until reserves are exhausted
field_pred_data_complete['reserve'] = field_pred_data_complete.groupby('field')['reserve'].ffill().fillna(0)
field_pred_data_complete['years_left'] = field_pred_data_complete.groupby('field')['years_left'].ffill().fillna(0)
field_pred_data_complete['predicted_production'] = field_pred_data_complete.groupby('field')['predicted_production'].ffill().fillna(0)
field_pred_data_complete['total_emissions'] = field_pred_data_complete.groupby('field')['total_emissions'].ffill().fillna(0)
field_pred_data_complete['mean_emission_intensity_distributed'] = field_pred_data_complete.groupby('field')['mean_emission_intensity_distributed'].ffill().fillna(0)
field_pred_data_complete['electrified'] = field_pred_data_complete.groupby('field')['electrified'].ffill().fillna(0)
field_pred_data_complete['cumulative_production'] = field_pred_data_complete.groupby('field')['cumulative_production'].ffill().fillna(0)
field_pred_data_complete['geometry'] = field_pred_data_complete.groupby('field')['geometry'].ffill().fillna('')

# When reserves are zero, set production and emissions to zero
field_pred_data_complete.loc[field_pred_data_complete['reserve'] == 0, ['predicted_production', 'total_emissions']] = 0

# Sort by field and year for readability
field_pred_data_complete = field_pred_data_complete.sort_values(by=["field", "year"]).reset_index(drop=True)

# Display the first few rows to verify
display(field_pred_data_complete.head(30))

,field,year,predicted_production,cumulative_production,total_emissions,years_left,reserve,mean_emission_intensity_distributed,electrified,geometry
0,aasta hansteen,2024,8.222942,8.222942,65186.450625,3.0,21.040000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
1,aasta hansteen,2025,8.222942,16.445884,68364.092286,2.0,12.817058,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
2,aasta hansteen,2026,4.594116,21.040000,45310.653241,1.0,4.594116,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
3,aasta hansteen,2027,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
4,aasta hansteen,2028,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
5,aasta hansteen,2029,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
6,aasta hansteen,2030,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
7,aasta hansteen,2031,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
8,aasta hansteen,2032,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."
9,aasta hansteen,2033,0.000000,21.040000,0.000000,0.0,0.000000,35.675739,0.0,"MULTIPOLYGON (((7.00225 67.04919, 7.00184 67.0..."


Let's refine the objective function to ensure a more logical weighting mechanism. We can adjust the trade-off parameter $\lambda$ so that it directly influences the balance between maximizing production and minimizing emissions.

Here's an updated formulation:

**Objective Function:**

$$ \text{maximize} \quad (1 - \lambda) \cdot \text{Production} - \lambda \cdot \text{Emissions} $$

where:
- $\lambda \in [0, 1]$ is the trade-off parameter.

If $\lambda = 0$:
- The objective maximizes production without considering emissions.

If $\lambda = 1$:
- The objective minimizes emissions without considering production.

This provides a more intuitive way to balance the two objectives.

**Variables:**
- $P_{f,y}$: Production of field $f$ in year $y$
- $E_{f,y}$: Emissions of field $f$ in year $y$
- $EL_{f,y}$: Electrification status of field $f$ in year $y$ (binary variable: 0 or 1)

**Constraints:**

1. **Emission Reduction Target for 2030:**

$$ \sum_{f \in F} E_{f,2030} \leq E_{2024} \times (1 - \text{target\_reduction}) $$

where:
- $E_{2024}$ is the baseline emissions for the year 2024.
- $\text{target\_reduction}$ is the emission reduction target (e.g., 55% by 2030).

2. **Production and Emission Relationships:**

$$ E_{f,y} = E_{f,y}^{\text{current}} + \text{coeff\_emissions\_elect} \cdot EL_{f,y} $$

where:
- $E_{f,y}^{\text{current}}$ is the current emission intensity.
- $\text{coeff\_emissions\_elect}$ is the coefficient representing the effect of electrification on emissions.

3. **Electrification Constraint:**

$$ EL_{f,y} \leq EL_{f,y-1} $$

This ensures that once a field is electrified, it remains electrified in subsequent years.

4. **Shut Down Constraint:**

$$ P_{f,y} = 0 \quad \forall \, y > Y_{\text{shut\_down}} $$

This ensures that once a field is shut down, it cannot be reopened in subsequent years.

**Objective Function Incorporating Electrification:**

The objective function now accounts for the adjustment due to electrification:

$$ \text{maximize} \quad (1 - \lambda) \cdot \sum_{f \in F} \sum_{y=2024}^{2050} P_{f,y} - \lambda \cdot \sum_{f \in F} \sum_{y=2024}^{2050} \left( E_{f,y}^{\text{current}} + \text{coeff\_emissions\_elect} \cdot EL_{f,y} \right) $$

where:
- $P_{f,y}$ is the production for field $f$ in year $y$.
- $E_{f,y}^{\text{current}}$ is the current emission intensity for field $f$ in year $y$.
- $\text{coeff\_emissions\_elect}$ is the negative coefficient from the regression representing the reduction in emissions due to electrification.